In [51]:
import csv
from pprint import pprint
### Formato CSV
def lee_fichero_accidentes(ruta):
    with open(ruta, 'r', newline='',encoding='utf8') as fich:
        ejemploLector = csv.DictReader(fich, delimiter=';')
        datos=list(ejemploLector)
    return datos

auxCSV=lee_fichero_accidentes("./AccidentesBicicletas_2021.csv")
print(type(auxCSV))
# for linea in auxCSV:
#     print(linea)

<class 'list'>


In [52]:
def contar_accidentes(datos):
    """
    Dada una lista de diccionarios de accidentes, devuelve un diccionario
    con clave tupla(distrito, tipo accidente) y valor cantidad de accidentes de ese tipo
    """
    contador = {}
    for accidente in datos:
        distrito = accidente.get('distrito')
        tipo_accidente = accidente.get('tipo_accidente')
        cantidad=contador.get((distrito,tipo_accidente),0)+1
        contador[(distrito, tipo_accidente)] = cantidad
    return contador

print(type(contar_accidentes(auxCSV)))

<class 'dict'>


In [ ]:
from collections import Counter
def dias_mas_accidentes(datos):
    """
    Dias con mas accidentes
    """
    auxsd={dato.get('fecha') for dato in datos}
    cantidad=Counter(auxsd).most_common()
    resultado={pareja for pareja in cantidad if pareja[1]==cantidad[0][1]}

    return resultado
print(dias_mas_accidentes(auxCSV))

In [53]:
def puntos_negros_distrito(datos, distrito, k):
    accidentes=[accidente['localizacion'] for accidente in datos if(accidente.get('distrito')==distrito)]
    contador=Counter(accidentes)
    repes=contador.most_common(k)
    repes.sort(key=lambda x:(x[1],x[0]),reverse=True)
    return repes

print(type(puntos_negros_distrito(auxCSV,'MONCLOA-ARAVACA',16)))

<class 'list'>


In [56]:
import json
import pprint
from collections import Counter
def leer_monumentos(ruta):
    with open(ruta, 'r',newline='',encoding='utf8')as fich:
        json_dict=json.load(fich)
    return json_dict["@graph"]
aux=leer_monumentos("300356-0-monumentos-ciudad-madrid.json")
print(type(aux))


<class 'list'>


In [59]:
def codigos_postales(monumentos):
    postal_code_monuments= [monumento["address"]["postal-code"] for monumento in monumentos ]
    return Counter(postal_code_monuments).most_common() 
aux_ej_2=codigos_postales(leer_monumentos("300356-0-monumentos-ciudad-madrid.json"))
print(type(aux_ej_2))
# for tupla in aux_ej_2:
#     print(tupla)

<class 'list'>


In [62]:

def busqueda_palabras_clave(monumentos, palabras):
    conjunto=set()
    for monumento in monumentos:
        words=True
        for palabra in palabras:
            desc_and_title=monumento["organization"]["organization-desc"]+" "+monumento["title"]
            if palabra.lower() not in desc_and_title.lower() :
                words=False
                break
        if(words):
            if "district" not in monumento["address"]:
                text=''
            else:
                text=monumento["address"]["district"]["@id"]
            conjunto.add((monumento["title"],text))
    return conjunto
aux_ej_3=busqueda_palabras_clave(aux,["escultura", "agua"])
print(type(aux_ej_3))
for tupla in aux_ej_3:
    print(tupla)

<class 'set'>
('Octaedro-Aire', 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Arganzuela')
('Fuentecilla del Leal', 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Centro')
('Felipe IV', 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Centro')
('Jacinto Verdaguer', 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Distrito')
('Puente de Toledo', 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Arganzuela')
('Fuente mural Dr. Jaime Ferrán y Clúa', 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Moncloa-Aravaca')
('Rubén Darío', 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Chamberi')
('Tetraedro-Fuego', 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Arganzuela')
('Fuente del pequeño tritón', 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madri

In [61]:
from geopy.geocoders import Nominatim
from geopy import distance
def busqueda_distancia(monumentos, direccion, distancia):
    geolocator = Nominatim(user_agent="GIW_pr2")
    lista=[]
    dir=geolocator.geocode(direccion,addressdetails=True)
    for monumento in monumentos:
        if "location" not in monumento:
            continue
        lugar=(monumento["location"]["latitude"],monumento["location"]["longitude"])
        distancia_al_monumento=distance.distance((dir.latitude,dir.longitude), lugar).km
        if distancia_al_monumento<distancia:
            lista.append((monumento["title"],monumento["id"],distancia_al_monumento))
            #Sacado de la documentación de sort si estan a la misma distancia mantiene el orden de aparición
            lista.sort(key=lambda monumento:monumento[2])
    return lista
aux_ej_4=busqueda_distancia(aux,"Profesor José García Santesmases 9, Madrid, España",1)
print(type(aux_ej_4))
# for terna in aux_ej_4:
#     print(terna)

<class 'list'>
